In [56]:
import pprint, re
from dateutil.parser import parse as dateParser
pprint.sorted = lambda x, key=None: x # disable sorting of results

In [2]:
import spacy
nlp = spacy.load("en_core_web_md") # load spacy English model

# Lab 07: Information Extraction

Student: John Wu

In [3]:
tagCap = re.compile(r'<P ID=(\d+)>\s+(.+?)\s?</P>', re.DOTALL)
def readFiles(filePath):
    with open(filePath, 'r', encoding='utf-8') as fh:
        matches = tagCap.findall(fh.read())
        ids,txts = zip(*matches)
        ids = [int(s) for s in ids]
        return ids, txts
    
trIDs, trTxts = readFiles('data/obits.train.txt') # get training files

# def firstTknAfterChar(doc, charOffset):
#     if charOffset >= len(doc.text): # if charOffset is after end of doc
#         return None
#     for tk in doc: # loop over tokens, find 1st token after offset
#         if tk.idx >= charOffset: # if token char-idx after offset
#             return tk # return token (contains char-idx and tkn-idx)

## Extraction of Required Relations

__Name of the deceased__

In [135]:
def extractName(doc):
    sent = next(doc.sents) # get first sentence of document
    for n,tk in enumerate(sent): # loop over tokens of first sentence
        if (tk.is_digit or tk.shape_[0]=='x' or tk.is_punct and 
                not (tk.text==',' or tk.is_left_punct or tk.is_right_punct)):
            break
    if doc[n-1].is_punct: # if span ends with punctuation, reduce span by 1
        n -= 1
    nameSpan = doc[:n]
    
    # see if any PERSON entity was matched in span
    if 'PERSON' not in set((tk.ent_type_ for tk in nameSpan)): # no PERSON
        for e in doc.ents: # loop over all extracted entities
            if e.label_ == 'PERSON': # find first PERSON entity
                nameSpan = e # set that entity to the name
                break
    return nameSpan.text, nameSpan.end

__Sex of the deceased__

In [5]:
def isMan(tokens):
    tokens = [tk.lower_ for tk in tokens]
    female = sum((tk=='her' or tk=='she' for tk in tokens))
    male = sum((tk=='his' or tk=='he' for tk in tokens))
    return male >= female

__Age at death__

In [144]:
ageMatch = spacy.matcher.Matcher(nlp.vocab)
ageMatch.add("age", None, [{"TEXT": {"REGEX": "^(1?\d\d)$"}}])
def getAgeDoc(doc, nameEnd=None):
    matches = ageMatch(doc)
    spanEnd = nameEnd if nameEnd else next(doc.sents).end

    numPos = list()
    for n,(nx,mBeg,mEnd) in enumerate(matches):
        if 0 < mBeg < spanEnd:
            return int(doc[mBeg:mEnd].text),mBeg
        if mBeg <= len(doc)//2 and doc[mBeg].ent_type_ != 'DATE':
            numPos.append( (int(doc[mBeg:mEnd].text),mBeg) )
    return max(numPos, key=lambda x: x[0]) if numPos else (None,None)

getAgeDoc(nlp(trTxts[15]),3)

(None, None)

__Location(s) of residency__

In [84]:
## note that first half of document likely contains most of the locations where
## the deceased live. The second half more likely to contain locations where
## the death happened and where the funeral service is held, which may not be
## the living location of the deceased.
###############################################################################

survivor = spacy.matcher.Matcher(nlp.vocab)
survivor.add("surv", None, [{"LOWER":{"REGEX": r'(surviv|pre-?deceas).*'}}])
locLikes = {'GPE', 'LOC', 'FAC', 'ORG', 'NORP'}

def findLocations(doc):
    part = doc[:len(doc)//10*8] # exclude the end (where funeral info are)
    locs,locIdxs = set(),set()
    
    for s in [doc[m[1]].sent for m in survivor(doc)]: # loop over sentences
        locIdxs.update( range(s.start,s.end+1) )
    
    for loc in (e for e in part.ents if (e.label_=='GPE' or e.label_=='LOC')):
        if loc.start in locIdxs:
            continue
        end = loc.end
        while ((doc[end].text==',' and doc[end+1].ent_type_ in locLikes) or 
               doc[end].ent_type_ in locLikes):
            end += 1
        locs.add(doc[loc.start:end].text)
        locIdxs.update( range(loc.start,end) )
    return locs # cannot find any GPE entities

{'Plymouth, Massachusetts', 'Sanibel', 'the Gulf of Mexico'}

In [98]:
# spacy.displacy.render(nlp(trTxts[6]), style='ent')

__Spouse(s) of the deceased__

In [110]:
# patterns like: survivors including his wife..., greatly missed by her husband...,
# predeceased by ...
# since many obits describe deceased as great husband/wife, must use gender to 
# distinguish descriptor of the deceased vs the spouse. Also gender is easy to find

spouses = spacy.matcher.Matcher(nlp.vocab)
spouseRe = r'(husband|wife|spouse|partner|married).*'
spouses.add('sp', None, [{"LOWER": {"REGEX":spouseRe}}] )

def findSpouseName(doc):
    matches = spouses(doc) # search for word related to spouses
    if not matches: # if no match, assumes no spouse can be found
        return None
    
    for x,mtBeg,mtEnd in matches:
        span = doc[mtBeg: doc[mtBeg].sent.end]
        for et in span.ents: # loop over all persons in text span
            if et.label_ == 'PERSON': 
                return et.text # return the first person found
    return None

## Extraction of Additional Relations

Three additional relations extracted from the text (if available) were: date of birth, date of death, and date of a funeral service

In [100]:
bdayMatch = spacy.matcher.Matcher(nlp.vocab)
bdayMatch.add("bday", None, [{"LOWER":"born"}],[{"LOWER":"birth"}])

deathMatch = spacy.matcher.Matcher(nlp.vocab)
deathSyns = r'^(die|pass|.?sleep|heaven|succumb|perish).*' # synonyms for death
deathMatch.add("death", None, [{"LOWER":{"REGEX": deathSyns}}])

funeralMatch = spacy.matcher.Matcher(nlp.vocab)
funeralMatch.add("celebration", None, [{"LOWER":"life"},{"LOWER":"celebration"}])
funeralMatch.add("serv", None, [{"LOWER":{"REGEX": r'^(service|visitation)s?'}}]) 
funeralMatch.add("memo", None, [{"LOWER":"memorial"}], [{"LOWER":"viewing"}])

################################################################################
def findDateAfterMatch(doc, matcher, startAtMatch=False):
    matched = matcher(doc)
    if not matched: # if no match, then nothing is after
        return None
    matchedSent = doc[matched[0][1]].sent # assume 1st match is good
    spanBeg = matched[0][1] if startAtMatch else matchedSent.start
    spanEnd = matchedSent.end
    for n in range(2):
        if spanEnd >= len(doc): # if no more sentences left in doc
            break
        spanEnd = doc[spanEnd].sent.end
        
    for et in doc[spanBeg:spanEnd].ents:
        if et.label_ == 'DATE' and re.findall(r'\d\d', et.text):
            return et.text
    return None

'Thursday, March 28, 2019'

## Filling Template and Outputting Result

(None, None)

In [126]:
def fillTemplate(doc):
    info = dict()
    info['name'],nameEnd = extractName(doc)
    info['sex'] = 'male' if isMan(doc) else 'female'
    info['age'] = getAgeDoc(doc)[0] # get age using document parsing
    info['locations'] = list(findLocations(doc))
    info['spouse'] = findSpouseName(doc)
    info['birth date'] = findDateAfterMatch(doc, bdayMatch)
    info['death date'] = findDateAfterMatch(doc, deathMatch, startAtMatch=True)
    info['funeral date'] = findDateAfterMatch(doc, funeralMatch, True)
################################################################################
    # try to calculate age at death using birth and death (or funeral) dates
    if info['birth date'] and (info['death date'] or info['funeral date']):
        bd = dateParser(info['birth date']) # parse birth date
        if info['death date']: # if death date is stated, parse death date
            dd = dateParser(info['death date']) 
        else: # otherwise use funeral service date as proxy for death
            dd = dateParser(info['funeral date'])
        elapsedYrs = (dd.year-bd.year) - ((dd.month,dd.day)<(bd.month,bd.day))
        if not info['age'] or abs(info['age'] - elapsedYrs)>2:
            info['age'] = elapsedYrs            
    
    return info

In [128]:
def parseObitsOutputInfo(obitFiles, outInfoFiles):
    docIDs,txts = readFiles(obitFiles)
    
    with open(outInfoFiles, 'w', encoding='utf-8') as outFH:
        for docID,txt in zip(docIDs,txts):
            doc = nlp(txt)
            out = {'ID': docID}
            out.update(fillTemplate(doc))
            pprint.pprint(out, outFH)
            outFH.write('\n')

In [129]:
parseObitsOutputInfo('data/obits.train.txt', 'obits.train2.out')

In [ ]:
# parseObitsOutputInfo('data/obits.test.txt', 'obits.test.out')

## Evaluation

In [32]:
dateutil.parser.parse('Sunday, January 13, 2019') >= dateutil.parser.parse('January 04, 2019')

True